In [1]:
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('~')
from analysts_tools.growth import *
    #Procurement tools
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

In [6]:
city = 'BOG'

In [2]:
def filter_customers(city, recency_day=20, kam=100, graduate=100, msgt=('ALL'), sgt=('ALL')):
    """
    This function filters the customers that have completed the specified criteria:
    city: The city that the customer has been registered -> i.e 'BOG'
    recency_day: The limit of days that the customer has completed an order -> 30 means 30 days or less
    kam: If the customer is a KAM or No -> {100: ALL, 1: True, 0: False}
    graduate: If the customer is GRADUATED or No -> {100: ALL, 1: True, 0: False}
    msgt: The microsegment of the customer, this is different based on the city
        - If you want to bring all should put as a tuple ('ALL')
        - If you want to bring one msgt then i.e ('Fonda')
        - If you want to bring more than one msgt then i.e ('Fonda','Corrientazo')
    sgt: The segment of the customer
        - If you want to bring all should put as a tuple ('ALL')
        - If you want to bring one msgt then i.e ('Restaurante')
        - If you want to bring more than one msgt then i.e ('Comercio','Restaurante')
    """
    import pandas as pd
    from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

    if isinstance(msgt, str):
        msgt = (msgt,'a')  # Convert the string to a tuple with a single element
    if isinstance(sgt, str):
        sgt = (sgt,'a')  # Convert the string to a tuple with a single element

    query = """
    SELECT
        source_id AS customer_id,
        is_kam,
        birthday,
        last_order_date,
        recency_days,
        registered_city AS region,
        microsegment_name AS microsegment,
        business_type AS segment,
        is_graduated
    FROM dpr_customer.obt_customer

    WHERE recency_days <= {days}
    ----------------------------------
    ----------------------------------
    AND 
    CASE
    WHEN '{city}' = 'ALL' THEN 1=1
    ELSE registered_city = '{city}'
    END
    ----------------------------------
    ----------------------------------
    AND
    CASE
    WHEN {is_kam} = 100 THEN 1=1
    ELSE is_kam = {is_kam}
    END
    ----------------------------------
    ----------------------------------
    AND
    CASE
    WHEN {graduated} = 100 THEN 1=1
    ELSE is_graduated = {graduated}
    END
    ----------------------------------
    ----------------------------------
    AND 
    CASE
    WHEN 'ALL' IN {microsegment} THEN 1=1
    ELSE microsegment_name IN {microsegment}
    END
    ----------------------------------
    ----------------------------------
    AND
    CASE
    WHEN 'ALL' IN {segment} THEN 1=1
    ELSE business_type IN {segment}
    END
    """.format(days=recency_day,city=city,is_kam=kam,graduated=graduate,microsegment=msgt,segment=sgt)

    return run_read_dwd_query(query)

In [3]:
def info_sells(city, sells_days_back_start=30, days_end=-1, customer=None):    
    """
    This returns a df with the information of the whole sells order by order based on the restrictions
    city: The city that the customer has been registered -> i.e 'BOG'
    sells_days_back_start: Since when we want to see the orders information
    days_end: Until when we want to see the orders information
    """    
    import pandas as pd
    from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
    import time
    import datetime
    
    sells_date_inicio = (datetime.datetime.today() - datetime.timedelta(days=sells_days_back_start)).strftime('%Y-%m-%d') #1 months
    sells_date_fin = (datetime.datetime.today() - datetime.timedelta(days=days_end)).strftime('%Y-%m-%d') #1 months
    
    tasas = {'BOG': 3776,
            'BAQ': 3776,
            'MDE': 3776,
            'CMX': 19.65,
            'GDL': 19.65,
            'PBC': 19.65,
            'SPO': 4.75,
            'BHZ': 4.75,
            'CWB': 4.75,
            'VCP': 4.75}

    if not isinstance(customer, list):
        raise TypeError("Customer parameter must be a list.")
    elif customer is None:    
        query = """
        SELECT 
            s.identifier_value AS region_code,
            DATE(fs.order_submitted_date) AS submit_date,
            fs.order_close_date AS close_date,
            fs.order_id,
            dc.source_id AS customer_id,
            cat.parent_description as category,
            cat.description AS subcategory,
            dp.card_id,
            dp.source_id AS sku_id,
            dp.card_description AS product_name,
            fs.product_quantity_x_step_unit AS cant,
            fs.gmv_pxq_local/{tasa_cambio} AS gmv_usd,
            COALESCE(fsd.product_discount/{tasa_cambio},0) AS discount_applied,
            fsd.adjustment_reason,
            COALESCE(bo.source_id,NULL) AS offer_id
            
        FROM dpr_sales.fact_sales                   fs
        INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
        INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
        INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
        INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
        LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
        LEFT JOIN dpr_sales.dim_offer               bo  ON bo.offer_id = fsd.dim_offer     

        WHERE 
            fs.gmv_enabled = TRUE
            AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
            AND fs.fb_order_status_id  IN (1,6,7,8)
            AND fs.is_deleted = FALSE
            AND fs.dim_status = 1
            AND dp.is_slot = 'false'
            --AND fsd.is_enabled = true
            AND s.identifier_value = '{ciudad}'
            AND DATE(fs.order_submitted_date) between '{Fecha_start}' AND '{Fecha_end}'
    """.format(tasa_cambio = tasas[city], ciudad = city, Fecha_start = sells_date_inicio, Fecha_end = sells_date_fin)
    else:
        query = """
        SELECT 
            s.identifier_value AS region_code,
            DATE(fs.order_submitted_date) AS submit_date,
            fs.order_close_date AS close_date,
            fs.order_id,
            dc.source_id AS customer_id,
            cat.parent_description as category,
            cat.description AS subcategory,
            dp.card_id,
            dp.source_id AS sku_id,
            dp.card_description AS product_name,
            fs.product_quantity_x_step_unit AS cant,
            fs.gmv_pxq_local/{tasa_cambio} AS gmv_usd,
            COALESCE(fsd.product_discount/{tasa_cambio},0) AS discount_applied,
            fsd.adjustment_reason,
            COALESCE(bo.source_id,NULL) AS offer_id
            
        FROM dpr_sales.fact_sales                   fs
        INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
        INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
        INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
        INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
        LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
        LEFT JOIN dpr_sales.dim_offer               bo  ON bo.offer_id = fsd.dim_offer     

        WHERE 
            fs.gmv_enabled = TRUE
            AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
            AND fs.fb_order_status_id  IN (1,6,7,8)
            AND fs.is_deleted = FALSE
            AND fs.dim_status = 1
            AND dp.is_slot = 'false'
            --AND fsd.is_enabled = true
            AND s.identifier_value = '{ciudad}'
            AND DATE(fs.order_submitted_date) between '{Fecha_start}' AND '{Fecha_end}'
            AND dc.source_id IN {clientes}
    """.format(tasa_cambio = tasas[city], ciudad = city, Fecha_start = sells_date_inicio, Fecha_end = sells_date_fin,clientes=tuple(set(customer)))   
    return run_read_dwd_query(query)

In [8]:
group = ('ALL')#,'Pratos/Refeições','Grill/Churrascaria/Espetinho','Padaria/Cafe/Bolos')

In [13]:
df_customers = filter_customers(city,60,100,100,group,('ALL')).customer_id.unique() #,'Taquería','Comida Rápida','Panadería/Pastelería'

In [14]:
df_customers

array([356307127, 356360311, 356316384, ...,  86975351, 343591007,
       270820727])

In [21]:
df_sells = info_sells(city,60,customer=df_customers.tolist())

In [24]:
customers = df_sells.loc[~df_sells.customer_id.isin(df_sells.loc[df_sells.card_id == 442768].customer_id.unique())].customer_id.unique().tolist()

In [33]:
#customers = df_sells[~df_sells.customer_id.isin(df_sells.loc[df_sells.card_id == 440828].customer_id.unique())].customer_id.unique()
customers = df_customers.tolist()

In [25]:
len(customers)

17293

In [26]:
list_as_string = str(customers)

# Open a text file in write mode
with open("leita.txt", "w") as file:
    # Write the list as a string to the file
    file.write(list_as_string)

In [27]:
import requests
import json

url = f"https://{city}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps([
    {
    "name": f"{city}_{group}_leita_SEP", 
    "customersIds": customers,
    "expiresAt": "2023-10-01"
}
    ]
                     )


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

Customer segments were created: [354622-BOG_ALL_leita_SEP]


In [36]:
import requests
import json

url = f"https://{city}.frubana.com/api/v1/segments/massive/update?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps({
  "customerSegmentId": 180701,
  "customersIds": customers
})
headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

The massive update process of Customer Segment was created
